## 講習3 --- 星の測光  

点光源の明るさを測定してみます。ここでは、IRAFのAPPHOTを用いて、アパーチャ測光を行います。  


#### 準備のトリミング  

講習2で作成した、btarget1n5.fitsをトリミングして、オーバースキャン領域など不要な部分を除いておきます。
(左端に明るい部分が見られます。これは何らかのバイアスがのっているものと思われるのでここ(25列目まで)も除きます。)

In [1]:
from pyraf import iraf

In [2]:
iraf.imcopy('btarget1n5.fits[25:2048, *]', 'btarget1n5trim.fits')

btarget1n5.fits[25:2048,*] -> btarget1n5trim.fits


### パラメータ設定  

iraf.apphot関連のタスクのためのパラメータを適切に設定する必要があります。 

#### 準備 
- 星のサイズ(fwhm)を求めておく
- 背景のレベルとばらつきをもとめておく

ds9を立ち上げておき、iraf.displayとiraf.imexamで星のfwhmを調べておきます。

このあとの作業では、'btarget1n5trim.fits'に対して繰り返し処理をおこなうので、

In [3]:
targetfits = 'btarget1n5trim.fits'

とファイル名を変数に代入しておきます。

In [4]:
iraf.display(targetfits)

z1=28.35372 z2=115.3357


In [5]:
iraf.imexam()

#   COL    LINE    COORDINATES
#     R    MAG    FLUX     SKY    PEAK    E   PA BETA ENCLOSED   MOFFAT DIRECT
1394.04  921.73 1394.04 921.73
  20.88  13.07  58984.   69.29   882.7 0.03   62 4.67     6.90     7.07   6.96
1384.47  706.26 1384.47 706.26
  21.55  12.25 125778.   69.22   1827. 0.03    9 15.8     6.97     7.37   7.18
 411.91 1136.10 411.91 1136.10
  21.58  12.33 116908.   69.13   1629. 0.06   23 4.76     7.20     7.66   7.19


fwhm=7.0 pixel としておきます。  

次に背景のmedianとノイズの評価をします。

In [6]:
iraf.unlearn('imstat')
iraf.imstat.fields = 'midpt, mean, stddev'
iraf.imstat.nclip = 3

In [7]:
iraf.imstat(targetfits)

#     MIDPT      MEAN    STDDEV
      68.84      69.1     7.368


メジアンは69でノイズは7.4とします。

#### パラメータの設定  

apphotモジュールをimportします。

In [8]:
from iraf import apphot

login.cl の記載内容によっては、
``` python 
from iraf import noao
from iraf import digiphot
``` 
も必要かもしれません。 

次に、測光に必要なパラメータを設定します。  
パラメータ設定の参考にした文献は、"A Reference Guide to the IRAF/DAOPHOT Package"です。  
http://iraf.noao.edu/docs/photom.html  
からダウンロード(daorefman.pdf)できます。  

In [9]:
med = 69.  #  背景レベルとばらつき、fwhm
std = 7.4
fwhm = 7.0

iraf.apphot.unlearn()  #  デフォルト値に戻しておく

iraf.apphot.datapars.datamax = 50000  #  サチった星を数えない
iraf.apphot.datapars.readnoise = 10 #  検出器に特有な値
iraf.apphot.datapars.epadu = 2.5 #  検出器に特有な値
iraf.apphot.datapars.itime = 10  # 積分時間 

iraf.apphot.findpars.threshold = 7  #  7シグマ以上のものを検出せよ 
iraf.apphot.findpars.sharphi = 0.8  # 星っぽくないものを除くため

#  fwhmで決まるパラメータ
iraf.apphot.datapars.fwhmpsf = fwhm   
iraf.apphot.centerpars.cbox = max(5.0, fwhm)
iraf.apphot.fitskypars.annulus = 3 * fwhm
iraf.apphot.photpars.apertures = 2 * fwhm

iraf.apphot.fitskypars.dannulus = 10.

#  背景のレベルとばらつきで決まるパラメータ
iraf.apphot.datapars.sigma = std  
iraf.apphot.datapars.datamin = med - 5 * std 

iraf.apphot.photpars.zmag = 27  # 等級のゼロ点

# IRAFと対話的(確認など)に行わないための設定
iraf.apphot.daofind.interac = 'no'
iraf.apphot.daofind.verify = 'no'
iraf.apphot.phot.interactive = 'no'  
iraf.apphot.phot.verify = 'no'
iraf.apphot.phot.verbose = 'no'

まずは、iraf.daofind()で星を検出させます。

In [10]:
iraf.daofind(targetfits, output='out1.coo')

どれが星として検出されたかFITS上にプロットしてみましょう。  
(線が細いので半径を3つ指定することで3つのマルを描く。)

In [11]:
iraf.tvmark('1', 'out1.coo', mark='circle', radii='15,16,17', color=207)

In [13]:
#iraf.help('tvmark')

星じゃないものも検出されちゃってますが、ここでは気にせず、daofindの出力の'out.coo'を
**iraf.phot()**に読み込ませて測光します。

In [14]:
iraf.phot(targetfits, coords='out1.coo', output='out1.mag')

最初の引数は、測光する対象のFITSファイル名です。coords=で読み込ませる座標ファイルの名前、output=で結果を書き出すファイル名を指定します。  

結果のファイルは、iraf.phot()固有の形式で書き出されています。

In [15]:
#cat 'out1.mag'   # githubでは長くなるのでコメントアウトしておきます

通常、必要なのは、xcenter, ycenter, mag, merr です。**iraf.txdump()**を使って、それらだけを抜き出します。

In [16]:
iraf.txdump('out1.mag', fields='xc,yc,mag,merr')

1384.487  706.380  16.778  0.003
1394.061  921.854  17.593  0.005
724.860  986.570  15.692  0.001
412.063  1136.186  16.854  0.003
1256.533  1254.923  19.872  0.032
1614.107  1296.000  24.509  2.293
1.975  1563.123  INDEF  INDEF
325.259  1710.853  20.233  0.047
295.760  1891.867  INDEF  INDEF
1214.740  1913.393  17.615  0.005
164.208  2090.797  20.722  0.073
1519.869  2131.413  19.975  0.035
755.494  2207.461  20.578  0.061
1609.915  2403.143  17.285  0.004
888.608  2443.525  21.548  0.153
400.753  2577.342  INDEF  INDEF
2008.147  2638.538  16.281  0.002
782.873  2801.123  18.178  0.008
1781.279  2834.660  18.671  0.012
261.980  2855.986  13.514  0.000
1841.312  2952.747  20.019  0.040
687.466  3180.978  13.860  0.001
752.111  3381.985  15.486  0.001
1741.726  4037.847  21.283  0.124
1333.792  4043.156  13.562  0.000
1791.647  4044.367  18.121  0.008
650.189  4085.125  19.898  0.038


INDEFなんてのもあります。これは、たまたまバッドピクセルが測光領域に含まれていた、視野の端である、あるいは、サチった星などです。
測光誤差を(例えば)0.2等以下のものだけに絞ることで、INDEFのものも削除できます。 誤検出のものもここで削除できます。

In [17]:
iraf.txdump('out1.mag', fields='xc,yc,mag,merr', expr='merr<0.2')

1384.487  706.380  16.778  0.003
1394.061  921.854  17.593  0.005
724.860  986.570  15.692  0.001
412.063  1136.186  16.854  0.003
1256.533  1254.923  19.872  0.032
325.259  1710.853  20.233  0.047
1214.740  1913.393  17.615  0.005
164.208  2090.797  20.722  0.073
1519.869  2131.413  19.975  0.035
755.494  2207.461  20.578  0.061
1609.915  2403.143  17.285  0.004
888.608  2443.525  21.548  0.153
2008.147  2638.538  16.281  0.002
782.873  2801.123  18.178  0.008
1781.279  2834.660  18.671  0.012
261.980  2855.986  13.514  0.000
1841.312  2952.747  20.019  0.040
687.466  3180.978  13.860  0.001
752.111  3381.985  15.486  0.001
1741.726  4037.847  21.283  0.124
1333.792  4043.156  13.562  0.000
1791.647  4044.367  18.121  0.008
650.189  4085.125  19.898  0.038


Stdout=''でファイル名を指定すると、そのファイルに書き出してくれます。

In [18]:
iraf.txdump('out1.mag', fields='xc,yc,mag,merr', expr='merr<0.2', Stdout='result1.txt')

これで、星の機械等級がもとまりました。  

### 測光値の較正  

ここで求まったものは、機械等級です。等級のゼロ点を適当に設定したものです。 
どうやって本当の等級に直せば良いでしょうか。  

実は、この視野には標準星が写っています。Landolt(1992)のカタログの標準星のうち以下が写っています。

<img src='./img/SA104.png', width='200'>  


| 標準星| カタログB等級(エラー) | 上記結果(機械等級)  | 3列 - 2列 | カタログB-V |  測定回数(カタログ) |   
| :- | :- | :- | :- | :- | :- |
|SA104-330 | 15.894 (0.029) | 17.593  (0.005) | 1.699 | 0.594 | 15 |  
|SA104-334 | 13.998 (0.006) | 15.692 (0.001) | 1.694 | 0.518 | 24 | 
|SA104-335 | 12.292 (0.010) | 13.860  (0.001) | 1.568 | 0.622 | 4   | 
|SA104-336 | 15.230 (0.010) | 16.854  (0.003) | 1.624 | 0.830 | 14 |   
|SA104-L2   | 16.700 (0.033) | 18.178  (0.008) | 1.478 | 0.650 | 4  |  

「3列 - 2列」の等級較正値にはばらつきがあります。 Landoltカタログで測定回数が10回以下のものは、ここでは、信頼性が低いとして採用しないことにします。 (それでもなお、SA104-336はSA104-330およびSA104-334と比べて0.07等も較正値が異なります。これはSA104-336のB-Vの値が他の2つと比べて大きいので、カラー変換の影響を考慮にいれなければいけないのかもしれません。)  

結果として、較正値の平均は1.672等、標準偏差は0.03等となりました。 
従って、result.txtの等級から1.672を引き、等級エラーには0.03等の誤差伝搬を加えておけばよいことになります。




In [19]:
import math 

with open('result1c.txt', 'w') as fout: # result_c.txtを書き出し先ファイルとする
    with open('result1.txt') as fin:  #  result.txtを開く 
        for line in fin:  #  1行づつ読み込み 
            v = line.rstrip().split()  #  rstrip()で改行コードを削除し、split()で空白文字で行を分割
            mag = float(v[2]) - 1.672 #  等級は3列目なので、それをfloatに変換して1.672を引く
            merr = math.sqrt(float(v[3])**2 + 0.03**2) 
            print (v[0], v[1], mag, merr, file=fout) #  書き出し

次の講習でとりあげる**numpy**を使うと、上のテキストファイル(result.txt)の読み出し、計算、書き出しのプログラムが非常に簡単になります。

### 演習3  

演習2-1で処理をした'btarget2n5.fits'で測光をしてみましょう。  
このときも、オーバースキャン領域などの不要な部分を削除して行いましょう。  

'btarget2n5.fits'の視野の中には測光標準星は写っていません。ただし、上のtarget1と近い時間に観測したデータですので、
等級ゼロ点は同じだと仮定し、上と同じ較正値(1.672)を使ってください。